<a href="https://colab.research.google.com/github/SheidaTalei/FinalProject/blob/main/SheidaTalei_BERT_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/drive


In [1]:
!pip install --target=$nb_path transformers

  Using cached https://files.pythonhosted.org/packages/f9/54/5ca07ec9569d2f232f3166de5457b63943882f7950ddfcc887732fc7fb23/transformers-4.3.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/71/23/2ddc317b2121117bf34dd00f5b0de194158f2a44ee2bf5e47c7166878a97/tokenizers-0.10.1-cp37-cp37m-manylinux2010_x86_64.whl
Processing /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45/sacremoses-0.0.43-cp37-none-any.whl


In [2]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
#Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import regex
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle
import re
import string
from transformers import TFBertModel, TFBertPreTrainedModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features, InputExample

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [ ]:
# %run SheidaTalei_Preprocessing_FinalProject.ipynb
# !pip install transformers

In [5]:
#------------------------------------------Empty Rows Removal----------------------------------------------------------------
# This function Will: 1- Remove all empty rows from csv file 2- Save data to the same csv
def removeEmptyRows(fileName):
    df = pd.read_csv(fileName, encoding='utf-8-sig')
    df = df.dropna(subset=['text'], how='all', axis=0) 
    df.to_csv(fileName, header=True, encoding='utf-8-sig',  index=False)
    
    return df

In [ ]:
# df1= removeEmptyRows ('/content/drive/MyDrive/Final/Prepared_Data_1_Labeled.csv')
# df2 =  removeEmptyRows ('/content/drive/MyDrive/Final/Prepared_train_Data_test#1.csv')




In [6]:
# X_and_Y = pd.concat([df1, df2], ignore_index=True, names=['tweeter_handle', 'text', 'Label', 'statues_count' ])
X_and_Y = removeEmptyRows ('/content/drive/MyDrive/Final/train_temp.csv')
train_data, valid_data = train_test_split(X_and_Y, test_size=0.3)
Y_train= train_data.Label
X_train = train_data.text


X_valid = valid_data.text
Y_valid = valid_data.Label

# data_frame= pd.concat([df1, df2, df3, df4, df5 ], ignore_index=True, names=['tweeter_handle', 'text', 'Label', 'statues_count' ])

In [7]:
count_0 = 0
count_1 = 0

for i in Y_valid:
    if (i==0):
        count_0 = count_0+1
    else:
        count_1 = count_1 + 1

In [8]:
print("count_0 ",count_0)
print("count_1 ",count_1)


count_0  6167
count_1  6378


In [9]:
count_0 = 0
count_1 = 0

for i in Y_train:
    if (i==0):
        count_0 = count_0+1
    else:
        count_1 = count_1 + 1

In [10]:
print("count_0 ",count_0)
print("count_1 ",count_1)

count_0  14542
count_1  14727


In [28]:
# X_and_Y_test = removeEmptyRows('/content/drive/MyDrive/Final/Prepared_test_Data_test#1.csv')
X_and_Y_test = removeEmptyRows('/content/drive/MyDrive/Final/test_temp.csv')
Y_test= X_and_Y_test.Label
X_test = X_and_Y_test.text

In [29]:
count_0 = 0
count_1 = 0

for i in Y_test:
    if (i==0):
        count_0 = count_0+1
    else:
        count_1 = count_1 + 1

In [30]:
print("count_0 ",count_0)
print("count_1 ",count_1)

count_0  5200
count_1  5254


In [ ]:
# X_and_Y_needLabel = removeEmptyRows('/content/drive/MyDrive/Final/Prepared_Data_1.csv')
# Y_needLabel= X_and_Y_needLabel.Label
# X_needLabel = X_and_Y_needLabel.text

In [ ]:
#source:
#https://curiousily.com/posts/sentiment-analysis-with-bert-and-hugging-face-using-pytorch-and-python/
#https://skimai.com/fine-tuning-bert-for-sentiment-analysis/
#https://sci2lab.github.io/ml_tutorial/bert_farsi_sentiment/

In [32]:
#We need to transform our data into a format BERT understands. 
# data is a dataFrame
def convert_data_into_input_example(data):
    """ Covert the list of examples into a list of `InputExample` objects that is suitable
        for BERT model."""
    input_examples = []
    for i in range(len(data)):
        example = InputExample(
            guid= None,
            text_a= data.iloc[i][ 'text'], #  is the text we want to classify
            text_b= None,
            label= data.iloc[i][ 'Label'] #is the label of our example, i.e. 1 or 0.
        )
        input_examples.append(example)
    return input_examples

In [33]:
train_input_examples = convert_data_into_input_example(X_and_Y)
val_input_examples = convert_data_into_input_example(valid_data)
#
# needLabel_input_example = convert_data_into_input_example(X_and_Y_needLabel)
test_input_examples = convert_data_into_input_example(X_and_Y_test)

In [34]:
train_input_examples[0]

InputExample(guid=None, text_a='یار طلبید#طلب مونس زیاد جان دولت حاجت ماست صحبت بس ان', text_b=None, label=0.0)

In [35]:

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

# text = 'I liked that book very much!'
# tokenized_text = tokenizer.tokenize(text)
# print(tokenized_text)
# text_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
# print('text ids:', text_ids)
# text_ids_with_special_tokens = tokenizer.build_inputs_with_special_tokens(text_ids)
# print('text ids with special tokens: ', text_ids_with_special_tokens)

In [36]:
#Tokenization
label_list = [0.0, 1.0]
MAX_SEQ_LENGTH = 150
bert_train_dataset = glue_convert_examples_to_features(examples=train_input_examples, tokenizer=tokenizer, max_length=MAX_SEQ_LENGTH, task='mrpc', label_list=label_list)
bert_val_dataset = glue_convert_examples_to_features(examples=val_input_examples, tokenizer=tokenizer, max_length=MAX_SEQ_LENGTH, task='mrpc', label_list=label_list)

#
bert_test_dataset =  glue_convert_examples_to_features(examples=test_input_examples, tokenizer=tokenizer, max_length=MAX_SEQ_LENGTH, task='mrpc', label_list=label_list)
# bert_needLabel_dataset =  glue_convert_examples_to_features(examples=needLabel_input_example, tokenizer=tokenizer, max_length=MAX_SEQ_LENGTH, task='mrpc', label_list=label_list)

# for i in range(3):
# #     print('Example: {}'.format(bert_train_dataset[i]))
#     print('Example: {')
#     print(' Input_ids: {}'.format(bert_train_dataset[i].input_ids))
#     print(' attention_mask: {}'.format(bert_train_dataset[i].attention_mask))
#     print(' token_type_ids: {}'.format(bert_train_dataset[i].token_type_ids))
#     print(' label: {}'.format(bert_train_dataset[i].label))
#     print('}')

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:175: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)


In [37]:
#for myself
ex = bert_train_dataset[0]
in_ids = ex.input_ids
decoded_sentence = tokenizer.decode(in_ids, skip_special_tokens=True)
print(decoded_sentence)

یار طلبید # طلب مونس زیاد جان دولت حاجت ماست صحبت بس ان


In [20]:
# import os
# checkpoint_path = "/content/drive/MyDrive/Final/Bert.ckpt"
# model = createLstmModel()
# model.load_weights(checkpoint_path)

In [38]:
#Defining the Hyperparameters
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels = 2)

# optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08, clipnorm=1.0)
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# metric_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# model.compile(optimizer=optimizer, loss=loss, metrics=[metric_accuracy , tf.keras.metrics.AUC()])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
#, tf.keras.metrics.AUC()
# model.add(tf.keras.layers.Dense(nb_classes, activation='sigmoid'))

model.compile(optimizer=optimizer, loss=loss, metrics=[ metric_accuracy])

In [40]:
#training the model
# model.fit(bert_train_dataset, validation_data=bert_val_dataset, epochs=3)
def my_solution(bdset):
    """ Create a list of input tensors required to be in the first argument of the
        model call function for training. e.g. `model([input_ids, attention_mask, token_type_ids])`.
    """
    input_ids, attention_mask, token_type_ids, label = [], [], [], []
    for in_ex in bdset:
        input_ids.append(in_ex.input_ids)
        attention_mask.append(in_ex.attention_mask)
        token_type_ids.append(in_ex.token_type_ids)
        label.append(in_ex.label)

    input_ids = np.vstack(input_ids)
    attention_mask = np.vstack(attention_mask)
    token_type_ids = np.vstack(token_type_ids)
    label = np.vstack(label)
    return ([input_ids, attention_mask, token_type_ids], label)
#Create a list of input tensors required to be in the first argument of the model call function for training. e.g.
# model([input_ids, attention_mask, token_type_ids]).

In [41]:
# Convert a training example into the Bert compatible format.
def example_to_features(input_ids, attention_masks, token_type_ids, y):
    """ Convert a training example into the Bert compatible format."""
    return {"input_ids": input_ids,
            "attention_mask": attention_masks,
            "token_type_ids": token_type_ids},y

In [25]:
#source: https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb#scrollTo=IFPuhwntH8VH
import os
checkpoint_path = "/content/drive/MyDrive/Final/Bert5.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint( filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [42]:
x_train, y_train = my_solution(bert_train_dataset)
x_val, y_val = my_solution(bert_val_dataset)

x_test, y_test = my_solution(bert_test_dataset)

#
# x_neededLabel, y_neededLabel = my_solution (bert_needLabel_dataset)

In [43]:
#This part helps for handeling imbalanced data
count_0 = 0
count_1 = 0

for i in y_train:
    if (i==0):
        count_0 = count_0+1
    else:
        count_1 = count_1 + 1

In [44]:
p = count_0 / (count_0 + count_1)

In [45]:
print(p)

0.4952647438656909


In [46]:
print('x_train shape: {}'.format(x_train[0].shape))
print('x_val shape: {}'.format(x_val[0].shape))

train_ds = tf.data.Dataset.from_tensor_slices((x_train[0], x_train[1], x_train[2], y_train)).map(example_to_features).shuffle(100).batch(32)
val_ds   = tf.data.Dataset.from_tensor_slices((x_val[0], x_val[1], x_val[2], y_val)).map(example_to_features).batch(64)
print(type (train_ds))
print('Format of model input examples: {} '.format(train_ds.take(1)))

#
test_ds =  tf.data.Dataset.from_tensor_slices((x_test[0], x_test[1], x_test[2], y_test)).map(example_to_features).batch(64)
# needLabel_ds = tf.data.Dataset.from_tensor_slices((x_neededLabel[0], x_neededLabel[1], x_neededLabel[2], y_neededLabel)).map(example_to_features).batch(64)

x_train shape: (41814, 150)
x_val shape: (12545, 150)
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
Format of model input examples: <TakeDataset shapes: ({input_ids: (None, 150), attention_mask: (None, 150), token_type_ids: (None, 150)}, (None, 1)), types: ({input_ids: tf.int64, attention_mask: tf.int64, token_type_ids: tf.int64}, tf.int64)> 


In [31]:




EPOCHS = 5 # 1000

# history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS , batch_size = 150)
# model.compile(optimizer=optimizer, loss=loss, metrics=[ metric_accuracy])
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS,  batch_size = 150, class_weight = {0: 1/p, 1: 1/(1-p)} ,callbacks=[cp_callback] ) #callbacks=[cp_callback],

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [ ]:
# train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], X_train))

In [ ]:
# train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))

In [ ]:
# from keras.preprocessing.sequence import pad_sequences
# train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")

In [ ]:

# tokens_tensor = torch.tensor([train_tokens_ids])
# Put the model in "evaluation" mode,meaning feed-forward operation.
# model.eval()

In [ ]:
# segments_ids = [1] * len(tokenized_text)
# print (segments_ids)
# tokens_tensor = torch.tensor([train_tokens_ids])
# segments_tensors = torch.tensor([segments_ids])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
# with torch.no_grad():
#     outputs = model(tokens_tensor,segments_tensors)

In [ ]:
# can use last hidden state as word embeddings
#source:https://medium.com/analytics-vidhya/bert-word-embeddings-deep-dive-32f6214f02bf
last_hidden_state = history[0]

In [47]:
#Loading the model
import os
checkpoint_path = "/content/drive/MyDrive/Final/Bert5.ckpt"

model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels = 2)
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')


model.compile(optimizer=optimizer, loss=loss, metrics=[ metric_accuracy])
model.load_weights(checkpoint_path)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [1]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [2]:
# model.fit(bert_train_dataset, validation_data=bert_val_dataset, epochs=3)
plot_history(history)

NameError: ignored

In [26]:
# predictions = model.predict(test_ds)
# print(predictions[0].shape)
# print()
# predictions_classes = np.argmax(predictions[0], axis = 1)
# for i in range(500):
#     print('text: {}\n, actual label: {}, predicted label: {}'.format(X_and_Y_test.iloc[i]['text'], val_input_examples[i].label, predictions_classes[i]))

In [ ]:
# def writeLabelInCsv(input_list,  fileName, lastFileName):
#   X_and_Y = pd.read_csv(fileName , encoding="utf-8")
#   counter = 34188
#   updator = 34188
#   for item in range(len(input_list)):
#     if counter <=80000:
#       X_and_Y.loc [counter, 'Label'] = input_list [item + updator] 
#       X_and_Y.to_csv(lastFileName ,index=False, encoding='utf-8-sig')
#       print( counter)
#       counter = counter + 1

#   print("counter: ", counter)


In [ ]:

#add label to datasets
prediction = model.predict(needLabel_ds)

In [ ]:
# load additional module
# import pickle

# # define a list of places

# # file_name = "/content/drive/MyDrive/Final/label_list_#1.pkl"
# # open_file = open(file_name, "wb")
# # pickle.dump(np.argmax(prediction[0], axis = 1), open_file)
# # open_file.close()

# file_name = "/content/drive/MyDrive/Final/label_list_#1.pkl"
# open_file = open(file_name, "rb")
# label_list = pickle.load(open_file)
# open_file.close()


In [ ]:
# writeLabelInCsv(label_list,  '/content/drive/MyDrive/Final/Prepared_Data_1.csv', '/content/drive/MyDrive/Final/Prepared_Data_1_Labeled.csv')
# import pandas as pd
# writeLabelInCsv(label_list,  '/content/drive/MyDrive/Final/Prepared_Data_1_Labeled.csv', '/content/drive/MyDrive/Final/Prepared_Data_1_Labeled.csv')

In [48]:
#newly added
predictions = model.predict(test_ds)
predictions_classes = np.argmax(predictions[0], axis = 1)
# print(predictions_classes)
print ('Accuracy for BERT: ',accuracy_score(y_test, predictions_classes))
print('F1-score for BERT: ', f1_score(y_test, predictions_classes))
print('roc_auc_score for BERT: ', roc_auc_score(y_test, predictions_classes))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
#Convert the test examples into Bert compatible format.
def example_to_features_predict(input_ids, attention_masks, token_type_ids):
    """
        Convert the test examples into Bert compatible format.
    """
    return {"input_ids": input_ids,
            "attention_mask": attention_masks,
            "token_type_ids": token_type_ids}


In [ ]:

def get_prediction(in_sentences):
    """
        Prepare the test comments and return the predictions.
    """
    label_list = [0.0, 1.0]
    input_examples = [InputExample(guid="", text_a = x, text_b = None, label = 0.0) for x in in_sentences] # here, "" is just a dummy label
    predict_input_fn = glue_convert_examples_to_features(examples=input_examples, tokenizer=tokenizer, max_length=MAX_SEQ_LENGTH, task='mrpc', label_list=label_list)
    x_test_input, y_test_input = my_solution(predict_input_fn)
    test_ds   = tf.data.Dataset.from_tensor_slices((x_test_input[0], x_test_input[1], x_test_input[2])).map(example_to_features_predict).batch(32)

    predictions = model.predict(test_ds)
    #   print('predictions:', predictions[0].shape)
    predictions_classes = np.argmax(predictions[0], axis = 1)
    return [(sentence, prediction) for sentence, prediction in zip(in_sentences, predictions_classes)]

In [ ]:
#Prepare the test comments and return the predictions

pred_sentences = []
i = 0
while(i<20):
  for tweet in X_and_Y_test.text:
    pred_sentences.append(tweet)
    i +=1
predictions = get_prediction(pred_sentences)
for p in predictions:
    print(p)

/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:175: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)


Streaming output truncated to the last 5000 lines.
('نمایشگاه حقوق بشر خارجه وزارت نقض امریکا سخنران ایران', 0)
('محکوم سه سالگی بازداشت شده بود زن… روز اعدام زندان شنبه مهرماه سیاسی', 0)
('مرد زن ریش اراستگی چسه کرد#کن اصلاح مو اقایون فرمی… جالبتره حالا', 0)
('دفن ایا سرباز ترین روستا میدانستید پربازدید نقاط ازادی ایران محل', 0)
('مربوط ظریف باید معرف مسئولای براشون جناب کشور مهم چینیا تجار میزدن مسائل اینجا نمیکنید رسید#رس امریکا مشنگ موقع روابط بال سفیر دیگه', 0)
('نوبت نشه فردا کثافت منه کاشت#کار جلو کدو می تونست دختر', 0)
('دردناک اشتیاق ازادی بردگی نداشته باشد برده', 0)
('شباهت علی نژاد ای مسیح معصومه مرور کرد#کن تاریخچه مزدور صاد… زیاد معروف قم مجاهدین', 0)
('باور اعمالشون همزمان تک چرت بگیره طرف برسن کثافتا اسید سزا دست دختره بنزین خانواده کرد#کن شوکر 😐 اینقدر چقدر بره چاقو جر تو نمیتونم راست اخه چندتا خونه بتونه دل', 0)
('بچه ستاره رفیق به ماه قدیم غریب میرسونن😉😅👌💙 فکر کن مشهد یهو بود#باش یاد سلام بیاد زنگ بزنی', 0)
('دختره مدیر بیشرف تجاوز مرد', 0)
('بیشتر دلار بدونید شاه شهی

In [ ]:
print("Evaluating the BERT model")
model.evaluate(test_ds)

Evaluating the BERT model
164/164 [==============================] - 102s 620ms/step - loss: 0.3813 - accuracy: 0.8599


[0.3812522292137146, 0.8598622679710388]